In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset

pip install 'aif360[LawSchoolGPA]'


In [2]:
raw_df=pd.read_csv("data.csv",index_col=0)
raw_df.drop_duplicates()
display(raw_df)

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,51552.0,C++;Python;Git;PostgreSQL,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,46482.0,Bash/Shell;HTML/CSS;JavaScript;Node.js;SQL;Typ...,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,77290.0,C;C++;Java;Perl;Ruby;Git;Ruby on Rails,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,46135.0,Bash/Shell;HTML/CSS;JavaScript;PHP;Ruby;SQL;Gi...,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,160932.0,C++;Python,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,<35,No,Undergraduate,1,Man,No,Dev,7,2,Germany,41058.0,C#;HTML/CSS;JavaScript;TypeScript;Docker;Kuber...,13,1
73458,>35,No,Undergraduate,1,Man,No,Dev,21,16,United States of America,115000.0,C#;HTML/CSS;Java;JavaScript;npm;ASP.NET Core ;...,11,1
73459,<35,No,Undergraduate,1,Man,No,Dev,4,3,Nigeria,57720.0,HTML/CSS;JavaScript;TypeScript;Docker;Express;...,12,1
73460,<35,Yes,Undergraduate,1,Man,Yes,Dev,5,1,United States of America,70000.0,C#;HTML/CSS;JavaScript;SQL;TypeScript;npm;Yarn...,15,1


In [ ]:
#Extract HaveWorkedWith column into several binary feature (Bag of Words Like)
computer_skill=pd.DataFrame()
feature_count=0
milestone=7346
startTime=time.time()
for index,row in raw_df.iterrows():
    hww_dict=dict()
    if(index==milestone):
        print(f"{milestone} entries have been examined,{round(time.time()-startTime)} seconds has passed since last report")
        startTime=time.time()
        milestone+=7346
    hww=row["HaveWorkedWith"]
    if hww is not np.nan:
        hww_list=str.split(hww,";")
        for ww in hww_list:
            column_name=ww
            if column_name not in computer_skill:
                computer_skill=pd.concat([computer_skill,pd.Series([np.nan]*index,name=column_name)],axis=1)
                feature_count+=1
            hww_dict[column_name]=1
        new_row=pd.DataFrame(hww_dict,index=[index])
    else:
        new_row=pd.Series([[np.nan]*feature_count],index=[index])
    computer_skill=pd.concat([computer_skill,new_row])
computer_skill=computer_skill.replace(np.nan,0)
display(computer_skill)
computer_skill.to_csv("computer_skill.csv")

In [ ]:
computer_skill=pd.read_csv("computer_skill.csv",index_col=0)

In [3]:
def min_max_normalisation(series):
    return (series - series.min()) / (series.max() - series.min())

rdf=raw_df.copy().drop("HaveWorkedWith",axis=1)
#normalize PreviousSalary feature
rdf["PreviousSalary"]=min_max_normalisation(rdf["PreviousSalary"])
#Create seperate dataframe to incorporate Computer Skill
#cs_rdf=pd.concat([rdf.drop(["Employed"],axis=1),computer_skill,rdf["Employed"]],axis=1)
display(rdf)
#display(cs_rdf)

,Age,Accessibility,EdLevel,Employment,Gender,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,ComputerSkills,Employed
0,<35,No,Master,1,Man,No,Dev,7,4,Sweden,0.230139,4,0
1,<35,No,Undergraduate,1,Man,No,Dev,12,5,Spain,0.207505,12,1
2,<35,No,Master,1,Man,No,Dev,15,6,Germany,0.345042,7,0
3,<35,No,Undergraduate,1,Man,No,Dev,9,6,Canada,0.205956,13,0
4,>35,No,PhD,0,Man,No,NotDev,40,30,Singapore,0.718445,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73457,<35,No,Undergraduate,1,Man,No,Dev,7,2,Germany,0.183291,13,1
73458,>35,No,Undergraduate,1,Man,No,Dev,21,16,United States of America,0.513391,11,1
73459,<35,No,Undergraduate,1,Man,No,Dev,4,3,Nigeria,0.257675,12,1
73460,<35,Yes,Undergraduate,1,Man,Yes,Dev,5,1,United States of America,0.312497,15,1


In [4]:
dataset=StandardDataset(df=rdf, label_name="Employed",
                                    favorable_classes=[1],
                                    scores_name="",
                                    protected_attribute_names=['Gender'],
                                    privileged_classes=[["Man"]],
                                    categorical_features=['Age','Accessibility','EdLevel','MentalHealth', 'MainBranch',"Country"])
#Encounter problem to convert cs_rdf to StandardDataset
# cs_dataset=StandardDataset(df=cs_rdf, label_name="Employed",
#                                     favorable_classes=[1],
#                                     scores_name="",
#                                     protected_attribute_names=['Gender'],
#                                     privileged_classes=[["Man"]],
#                                     categorical_features=['Age','Accessibility','EdLevel','MentalHealth', 'MainBranch',"Country"])

In [5]:
#Train, Test set splits of original Datasets
train_set, test_set = dataset.split([0.7], shuffle=True)


#Train, Test set splits of Datasets with individual computer skills information
# cs_train_set, cs_test_set = train_test_split(cs_df,test_size=0.3,train_size=0.7)


# cs_train_X=train_set.drop(["Employed"],axis=1)
# cs_train_y=train_set["Employed"]

# cs_test_X=test_set.drop(["Employed"],axis=1)
# cs_test_y=test_set["Employed"]

In [6]:
merics_data_column=["Accuracy","Disparate Impact","Disparate Parity","Equal Opportunity (TPR difference)","Equalized Odds (FPR difference)"]
#merics_data=pd.DataFrame(columns=merics_data_column)
merics_data=pd.read_csv("Benchmark_data.csv",index_col=0)

privileged_groups = [{"Gender":1}]
unprivileged_groups = [{"Gender":0}]

display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577
DisparateImpactRemover (0.5),0.772222,0.981002,-0.009710,0.035720,0.018005
DisparateImpactRemover (0.7),0.746359,1.215998,0.089430,0.140702,0.107119
DisparateImpactRemover (1.0),0.739190,1.310888,0.123178,0.176847,0.137168
EqOddsPostprocessing,0.774264,0.886463,-0.063059,-0.010003,-0.056543


In [8]:
#Train RandomForest as Baseline model on original Datasets
baseline=RandomForestClassifier()
baseline.fit(train_set.features,train_set.labels.ravel())

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [9]:
pred=baseline.predict(test_set.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.labels=pred
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
baseline_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([baseline_metric],index=["Random Forest"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.77254,0.872836,-0.071735,-0.037152,-0.034956


In [10]:
#Train Adversial Debiasing on original Datasets
import tensorflow.compat.v1 as tf
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

tf.disable_eager_execution()
sess = tf.Session()
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)
debiased_model.fit(train_set)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.829179; batch adversarial loss: 1.529383
epoch 0; iter: 200; batch classifier loss: 0.986719; batch adversarial loss: 1.230312
epoch 0; iter: 400; batch classifier loss: 0.713100; batch adversarial loss: 0.917078
epoch 1; iter: 0; batch classifier loss: 0.732247; batch adversarial loss: 0.940207
epoch 1; iter: 200; batch classifier loss: 0.644840; batch adversarial loss: 0.725584
epoch 1; iter: 400; batch classifier loss: 0.547088; batch adversarial loss: 0.556616
epoch 2; iter: 0; batch classifier loss: 0.407871; batch adversarial loss: 0.537930
epoch 2; iter: 200; batch classifier loss: 0.482400; batch adversarial loss: 0.456945
epoch 2; iter: 400; batch classifier loss: 0.499467; batch adversarial loss: 0.461823
epoch 3; iter: 0; batch classifier loss: 0.461412; batch adversarial loss: 0.401525
epoch 3; iter: 200; batch classifier loss: 0.447901; batch adversarial loss: 0.366033
epoch 3; iter: 400; batch classifier loss: 0.496869; batch adv

In [11]:
test_pred=debiased_model.predict(test_set)
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
adversial_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([adversial_metric],index=["Adversial Debias"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485


In [12]:
from aif360.algorithms.inprocessing import GridSearchReduction

gsr_eo_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="EqualizedOdds",prot_attr="Gender")
gsr_eo_model.fit(train_set)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
test_pred=gsr_eo_model.predict(test_set)
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
gsr_eo_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([gsr_eo_metric],index=["GridSearchReduction(EO)"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967


In [19]:
from aif360.algorithms.inprocessing import GridSearchReduction

gsr_dp_model=GridSearchReduction(estimator=RandomForestClassifier(),constraints="DemographicParity",prot_attr="Gender")
gsr_dp_model.fit(train_set)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
test_pred=gsr_dp_model.predict(test_set)
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
gsr_dp_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([gsr_dp_metric],index=["GridSearchReduction(DP)"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482


In [35]:
from aif360.algorithms.preprocessing import Reweighing

reweight=Reweighing(privileged_groups=privileged_groups,unprivileged_groups=unprivileged_groups)
reweight.fit(train_set)
reweight_set=reweight.transform(train_set.copy(deepcopy=True))

reweight_model=RandomForestClassifier()
reweight_model.fit(reweight_set.features,reweight_set.labels.ravel(),sample_weight=train_set.instance_weights)

RandomForestClassifier()

In [37]:
reweight_test = reweight.transform(test_set.copy(deepcopy=True))
pred=reweight_model.predict(reweight_test.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.labels=pred
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
reweight_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([reweight_metric],index=["Reweight"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577


In [43]:
from aif360.algorithms.preprocessing import DisparateImpactRemover

di = DisparateImpactRemover(repair_level=0.5,sensitive_attribute="Gender")
train_di = di.fit_transform(train_set.copy(deepcopy=True))
test_di = di.fit_transform(test_set.copy(deepcopy=True))

di_model=RandomForestClassifier()
di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)


RandomForestClassifier()

In [44]:
pred=di_model.predict(test_di.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.labels=pred
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
di_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=merics_data.drop(["DisparateImpactRemover (0.7)"])
merics_data=pd.concat([merics_data,pd.DataFrame([di_metric],index=["DisparateImpactRemover (0.5)"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577
DisparateImpactRemover (0.5),0.772222,0.981002,-0.009710,0.035720,0.018005


In [45]:
di = DisparateImpactRemover(repair_level=0.7,sensitive_attribute="Gender")
train_di = di.fit_transform(train_set.copy(deepcopy=True))
test_di = di.fit_transform(test_set.copy(deepcopy=True))

di_model=RandomForestClassifier()
di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)

RandomForestClassifier()

In [46]:
pred=di_model.predict(test_di.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.labels=pred
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
di_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([di_metric],index=["DisparateImpactRemover (0.7)"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577
DisparateImpactRemover (0.5),0.772222,0.981002,-0.009710,0.035720,0.018005
DisparateImpactRemover (0.7),0.746359,1.215998,0.089430,0.140702,0.107119


In [47]:
di = DisparateImpactRemover(repair_level=1,sensitive_attribute="Gender")
train_di = di.fit_transform(train_set.copy(deepcopy=True))
test_di = di.fit_transform(test_set.copy(deepcopy=True))

di_model=RandomForestClassifier()
di_model.fit(train_di.features,train_di.labels.ravel(),sample_weight=train_di.instance_weights)

RandomForestClassifier()

In [48]:
pred=di_model.predict(test_di.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.labels=pred
metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
di_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([di_metric],index=["DisparateImpactRemover (1.0)"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577
DisparateImpactRemover (0.5),0.772222,0.981002,-0.009710,0.035720,0.018005
DisparateImpactRemover (0.7),0.746359,1.215998,0.089430,0.140702,0.107119
DisparateImpactRemover (1.0),0.739190,1.310888,0.123178,0.176847,0.137168


In [13]:
from aif360.algorithms.postprocessing import CalibratedEqOddsPostprocessing

eop_model=RandomForestClassifier()
eop_model.fit(train_set.features,train_set.labels.ravel())

train_eop_set=train_set.copy(deepcopy=True)
train_eop_set.scores=eop_model.predict(train_set.features)

eop=CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
eop.fit(train_set,train_eop_set)


In [15]:
pred=eop_model.predict(test_set.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.scores=pred
test_pred=eop.predict(test_pred)

metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
eop_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([eop_metric],index=["EqOddsPostprocessing"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577
DisparateImpactRemover (0.5),0.772222,0.981002,-0.009710,0.035720,0.018005
DisparateImpactRemover (0.7),0.746359,1.215998,0.089430,0.140702,0.107119
DisparateImpactRemover (1.0),0.739190,1.310888,0.123178,0.176847,0.137168
EqOddsPostprocessing,0.774264,0.886463,-0.063059,-0.010003,-0.056543


In [7]:
from aif360.algorithms.postprocessing import RejectOptionClassification

roc_dp_model=RandomForestClassifier()
roc_dp_model.fit(train_set.features,train_set.labels.ravel())

train_roc_dp_set=train_set.copy(deepcopy=True)
train_roc_dp_set.scores=roc_dp_model.predict(train_set.features)

roc_dp = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                  metric_name="Statistical parity difference")
roc_dp.fit(train_set, train_roc_dp_set)

c:\Users\15783\source\repos\Fairness-Metrics-Research\venv\Lib\site-packages\aif360\algorithms\postprocessing\reject_option_classification.py:160: UserWarning: Unable to satisy fairness constraints
  warn("Unable to satisy fairness constraints")


In [12]:
pred=roc_dp_model.predict(test_set.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.scores=pred
test_pred=roc_dp.predict(test_pred)

metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
roc_dp_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([roc_dp_metric],index=["RejectOptionClassification(DP)"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577
DisparateImpactRemover (0.5),0.772222,0.981002,-0.009710,0.035720,0.018005
DisparateImpactRemover (0.7),0.746359,1.215998,0.089430,0.140702,0.107119
DisparateImpactRemover (1.0),0.739190,1.310888,0.123178,0.176847,0.137168
EqOddsPostprocessing,0.774264,0.886463,-0.063059,-0.010003,-0.056543
RejectOptionClassification(DP),0.776215,0.833459,-0.094213,-0.036742,-0.066124


In [9]:
roc_EO_model=RandomForestClassifier()
roc_EO_model.fit(train_set.features,train_set.labels.ravel())

train_roc_EO_set=train_set.copy(deepcopy=True)
train_roc_EO_set.scores=roc_EO_model.predict(train_set.features)

roc_EO = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                  metric_name="Equal opportunity difference")
roc_EO.fit(train_set, train_roc_EO_set)

In [13]:
pred=roc_EO_model.predict(test_set.features)
test_pred=test_set.copy(deepcopy=True)
test_pred.scores=pred
test_pred=roc_EO.predict(test_pred)

metric=ClassificationMetric(test_set,test_pred,unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
roc_EO_metric=[metric.accuracy(),metric.disparate_impact(),metric.statistical_parity_difference(),metric.true_positive_rate_difference(),metric.false_positive_rate_difference()]
merics_data=pd.concat([merics_data,pd.DataFrame([roc_EO_metric],index=["RejectOptionClassification(EO)"],columns=merics_data_column)])
display(merics_data)

,Accuracy,Disparate Impact,Disparate Parity,Equal Opportunity (TPR difference),Equalized Odds (FPR difference)
Random Forest,0.772540,0.872836,-0.071735,-0.037152,-0.034956
Adversial Debias,0.782885,0.900093,-0.057727,-0.012537,-0.028485
GridSearchReduction(EO),0.773356,0.903638,-0.054205,-0.014497,-0.021967
GridSearchReduction(DP),0.772630,0.909401,-0.050985,-0.009628,-0.020482
Reweight,0.771677,0.869303,-0.073716,-0.037619,-0.038577
DisparateImpactRemover (0.5),0.772222,0.981002,-0.009710,0.035720,0.018005
DisparateImpactRemover (0.7),0.746359,1.215998,0.089430,0.140702,0.107119
DisparateImpactRemover (1.0),0.739190,1.310888,0.123178,0.176847,0.137168
EqOddsPostprocessing,0.774264,0.886463,-0.063059,-0.010003,-0.056543
RejectOptionClassification(DP),0.776215,0.833459,-0.094213,-0.036742,-0.066124


In [14]:
merics_data.to_csv("Benchmark_data.csv")